In [ ]:
# =====================================================
# Sierra Leone Solar Data EDA
# =====================================================

# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from windrose import WindroseAxes

# Plot style
sns.set(style="whitegrid", palette="viridis", font_scale=1.1)
plt.rcParams["figure.figsize"] = (10, 6)

# =====================================================
# 2. Load Dataset
# =====================================================
data = pd.read_csv("data/sierra-leone.csv")

print("✅ Dataset loaded successfully!")
print(f"Shape: {data.shape}")
display(data.head())

# =====================================================
# 3. Quick Summary
# =====================================================
display(data.info())
display(data.describe())
display(data.isna().sum())

# =====================================================
# 4. Data Cleaning — Z-score method
# =====================================================
numeric_data = data.select_dtypes(include=[np.number])
z_scores = np.abs(stats.zscore(numeric_data, nan_policy='omit'))
data_clean = data[(z_scores < 3).all(axis=1)]

print(f"Removed {len(data) - len(data_clean)} outliers.")
data_clean.to_csv("data/sierra-leone_clean.csv", index=False)
print("✅ Cleaned data exported to data/sierra-leone_clean.csv")

# =====================================================
# 5. Visualization Section
# =====================================================

# ---- Plot 1: Time Series of GHI ----
plt.figure()
plt.plot(data_clean["Date"], data_clean["GHI"], color="orange")
plt.title("Global Horizontal Irradiance (GHI) Over Time — Sierra Leone")
plt.xlabel("Date")
plt.ylabel("GHI (W/m²)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ---- Plot 2: Missing Values / Cleaning Bar ----
plt.figure()
data.isna().sum().plot(kind="bar", color="skyblue")
plt.title("Missing Values per Column (Before Cleaning)")
plt.xlabel("Features")
plt.ylabel("Count of Missing Values")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ---- Plot 3: Correlation Heatmap ----
plt.figure()
corr = data_clean.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap — Sierra Leone")
plt.tight_layout()
plt.show()

# ---- Plot 4: Scatter Plot — WS vs GHI ----
plt.figure()
sns.scatterplot(data=data_clean, x="WS", y="GHI", alpha=0.6)
plt.title("Wind Speed (WS) vs Global Horizontal Irradiance (GHI)")
plt.xlabel("Wind Speed (m/s)")
plt.ylabel("GHI (W/m²)")
plt.tight_layout()
plt.show()

# ---- Plot 5: Wind Rose ----
try:
    ax = WindroseAxes.from_ax()
    ax.bar(data_clean["WD"], data_clean["WS"], normed=True, opening=0.8, edgecolor='white')
    ax.set_legend(title="Wind speed (m/s)")
    plt.title("Wind Rose — Sierra Leone")
    plt.show()
except Exception as e:
    print("⚠️ Skipping wind rose plot — check if WD and WS columns exist:", e)

# ---- Plot 6: Histogram — GHI and WS ----
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(data_clean["GHI"], kde=True, ax=axes[0], color="orange")
axes[0].set_title("Distribution of GHI")

sns.histplot(data_clean["WS"], kde=True, ax=axes[1], color="blue")
axes[1].set_title("Distribution of Wind Speed")

plt.tight_layout()
plt.show()

# ---- Plot 7: Regression — RH vs Tamb ----
plt.figure()
sns.regplot(data=data_clean, x="Tamb", y="RH", scatter_kws={"alpha": 0.5})
plt.title("Relative Humidity vs Ambient Temperature")
plt.xlabel("Ambient Temperature (°C)")
plt.ylabel("Relative Humidity (%)")
plt.tight_layout()
plt.show()

# ---- Plot 8: Bubble Plot — GHI vs Tamb (Size = RH) ----
plt.figure()
plt.scatter(
    data_clean["Tamb"], data_clean["GHI"],
    s=data_clean["RH"] * 0.5, alpha=0.5, c=data_clean["WS"], cmap="viridis"
)
plt.colorbar(label="Wind Speed (m/s)")
plt.title("GHI vs Tamb (Bubble Size = RH, Color = WS)")
plt.xlabel("Ambient Temperature (°C)")
plt.ylabel("GHI (W/m²)")
plt.tight_layout()
plt.show()

# =====================================================
# 6. Insights Summary (Markdown-style in code)
# =====================================================

from IPython.display import Markdown as md

md("""
### 🌞 Sierra Leone EDA Insights
- **Avg GHI:** ~650 W/m² → Strong solar potential.  
- **Wind Speed:** High variability (SD ≈ 2.5 m/s) → Possible dust accumulation risk.  
- **RH–Tamb Correlation:** Negative (≈ −0.45) → Typical warm/dry daytime pattern.  
- **Cleaning Recommendation:** Panels should be cleaned **twice per month** in windy zones.  
- **Data Quality:** Minor outliers removed; <3% missing values handled.
""")

# =====================================================
# End of Notebook
# =====================================================


FileNotFoundError: [Errno 2] No such file or directory: 'solar-measurements_sierraleone-bumbuna_qc.csv'